<a href="https://colab.research.google.com/github/Imran0897/Fake-News-Classifier-Using-LSTM/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploades = files.upload()   

Saving train.csv to train (1).csv


In [ ]:
import tensorflow

In [ ]:
import pandas as pd
df = pd.read_csv('train.csv',on_bad_lines = 'skip',encoding="latin1")

In [ ]:
df

,id,title,author,text,label
0,0.0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1.0
1,1.0,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2.0,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3.0,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4.0,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0
...,...,...,...,...,...
2074,2107.0,"Maddow Raises Possibility Trump, Campaign Inno...",Jeff Poor,Thursday on MSNBCâs âThe Rachel Maddow Sho...,0.0
2075,2108.0,Last Days of John Kirby: Hapless State Rep Get...,NaN,This post was originally published on this sit...,1.0
2076,2109.0,Big Expose! Why Mamata Banarjee is attacking M...,NaN,"Viswajith | November 15, 2016 | Viral Videos |...",1.0
2077,2110.0,A Struggling Theme Park Asks: Do You Still Bel...,Annie Stoltie,"NORTH POLE, N. Y. â On a snowy shoulder o...",0.0


In [ ]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [ ]:
df.head()

,id,title,author,text,label
0,0.0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1.0
1,1.0,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2.0,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3.0,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4.0,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [ ]:
df =df.dropna()

In [ ]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
X = df.drop(columns=['label'])

In [ ]:
y = df['label']

In [ ]:
X.shape

(1785, 4)

In [ ]:
y.shape

(1785,)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size = 5000

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace = True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hillari',
 'chuck todd buzzfe donald trump polit favor breitbart',
 'monica lewinski clinton sex scandal set american crime stori',
 'rob reiner trump mental unstabl breitbart',
 'abort pill order rise latin american nation zika alert new york

In [ ]:
onehot_repr = [one_hot(words , voc_size) for words in corpus]
onehot_repr

[[4616, 1425, 3922, 3005, 4536, 2618, 3363, 1756, 2288, 2966],
 [4475, 2388, 1018, 2577, 1855, 3044, 3021],
 [4294, 4854, 2825, 3206],
 [4902, 4857, 4755, 3368, 1968, 2521],
 [4266, 1855, 2635, 3699, 2276, 111, 1855, 2877, 3142, 1845],
 [1098,
  674,
  3723,
  4659,
  4040,
  1909,
  252,
  653,
  2471,
  2198,
  361,
  4891,
  3776,
  853,
  3021],
 [4170, 4331, 1570, 4463, 2867, 1744, 548, 2007, 1734, 569, 4675],
 [2443, 3172, 299, 3965, 3765, 768, 1909, 1086, 1734, 569, 4675],
 [2014, 439, 1820, 1820, 3152, 2983, 379, 4536, 1909, 2141],
 [4497, 935, 2850, 1353, 3318, 3154, 4563, 1072],
 [4180, 599, 1706, 1380, 3141, 1734, 569, 4675],
 [2618, 2800, 1691, 2055, 2225, 3698, 265, 43, 1479, 4267],
 [3637, 336, 2388],
 [1500, 3776, 1585, 1442, 1909, 1204, 3053, 3021],
 [4955, 2216, 1018, 3075, 214, 3188, 1978, 4268, 111],
 [4306, 3889, 1909, 2110, 246, 3021],
 [823, 4767, 3515, 1912, 1619, 1978, 918, 112, 250, 1734, 569, 4675],
 [4144, 1604, 2969, 1208, 2423, 4450, 1259],
 [3776, 1213, 45

In [ ]:
from nltk.lm.preprocessing import pad_sequence
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding= 'pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1756 2288 2966]
 [   0    0    0 ... 1855 3044 3021]
 [   0    0    0 ... 4854 2825 3206]
 ...
 [   0    0    0 ... 1636  252 3021]
 [   0    0    0 ... 1734  569 4675]
 [   0    0    0 ... 1734  569 4675]]


In [ ]:
len(embedded_docs)

1785

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_final , y_final , test_size = 0.3 , random_state=144)


In [ ]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 10 , batch_size = 64)

Epoch 1/10
20/20 [==============================] - 7s 30ms/step - loss: 0.6652 - accuracy: 0.5941 - val_loss: 0.6139 - val_accuracy: 0.7761
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5135 - accuracy: 0.7958 - val_loss: 0.4854 - val_accuracy: 0.8041
Epoch 3/10
20/20 [==============================] - 0s 9ms/step - loss: 0.3072 - accuracy: 0.8831 - val_loss: 0.3899 - val_accuracy: 0.8209
Epoch 4/10
20/20 [==============================] - 0s 8ms/step - loss: 0.1473 - accuracy: 0.9512 - val_loss: 0.4024 - val_accuracy: 0.8358
Epoch 5/10
20/20 [==============================] - 0s 8ms/step - loss: 0.0660 - accuracy: 0.9816 - val_loss: 0.3820 - val_accuracy: 0.8358
Epoch 6/10
20/20 [==============================] - 0s 8ms/step - loss: 0.0421 - accuracy: 0.9920 - val_loss: 0.4127 - val_accuracy: 0.8507
Epoch 7/10
20/20 [==============================] - 0s 10ms/step - loss: 0.0265 - accuracy: 0.9960 - val_loss: 0.5847 - val_accuracy: 0.8228
Epoch 8/10
20/20 [

In [ ]:
from tensorflow.keras.layers import Dropout

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
y_pred=model.predict(X_test)

17/17 [==============================] - 0s 2ms/step


In [ ]:
y_pred = np.where(y_pred>0.5 , 1,0)

In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score

In [ ]:
confusion_matrix(y_test,y_pred)

array([[234,  68],
       [150,  84]])

In [ ]:
accuracy_score(y_test,y_pred)

0.5932835820895522